# Universidad Autónoma de Yucatán

## Facultad de Matemáticas

### Machine Learning

**Teacher:** Dr. Victor Uc Cetina <[victoruccetina@gmail.com](mailto:victoruccetina@gmail.com)>

**Student:** Dayan Bravo Fraga <[dayan3847@gmail.com](mailto:dayan3847@gmail.com)>

# Gradient Descent: Polynomial **Gaussian** Sigmoidal

[GitHub](https://github.com/dayan3847/machine_learning/blob/master/dayan3847/stochastic_gradient_descent_gaussian/gaussian.ipynb)

[Colab](https://colab.research.google.com/github/dayan3847/machine_learning/blob/master/dayan3847/stochastic_gradient_descent_gaussian/gaussian.ipynb)

In [132]:
import numpy as np
import sympy as sp
import plotly as py
from abc import ABC, abstractmethod

In [133]:
points_number: int = 100

In [134]:
gen_function: sp.Expr = sp.sin(2 * sp.pi * sp.Symbol('x'))
gen_function

sin(2*pi*x)

In [135]:
def generate_data() -> np.array:
    _x: np.array = np.random.rand(points_number)
    _e: np.array = np.random.rand(points_number) * .6 - .3
    _y: np.array = np.sin(2 * np.pi * _x) + _e
    return np.array([_x, _y])


my_data: np.array = generate_data()
my_data.shape

(2, 100)

In [136]:
def get_plot_functions() -> dict:
    _x = np.linspace(0, 1, 1000)
    _yg = np.sin(2 * np.pi * _x)
    return {
        'x': _x,
        'sin': _yg,
    }


fun = get_plot_functions()
fun['x'].shape

(1000,)

In [137]:
fig = py.graph_objs.Figure()
fig.add_scatter(x=fun['x'], y=fun['sin'], name=str(gen_function), line_dash='dash', line_color='gray')
fig.add_scatter(x=my_data[0], y=my_data[1], name='Points', mode='markers', marker_color='red')
fig.show()

In [138]:
class Model(ABC):
    def __init__(self, data: np.array):
        self.c: int = 9  # Number of basis functions
        self.w = np.random.rand(self.c) - .5  # Weights
        self.data: np.array = data  # Data
        self.epochs: int = 40  # Number of epochs
        self.a: float = 0.01  # Learning rate

        self.error_history: np.array = np.array([])  # Error history

    @abstractmethod
    def equation_basis_function(self) -> sp.Expr:
        pass

    @abstractmethod
    def basis_function(self, x: float) -> np.array:
        pass

    @abstractmethod
    def equation(self) -> sp.Expr:
        pass

    # Calculate the model value for a simple value
    def hi(self, x: float) -> float:
        bf: np.array = self.basis_function(x)
        return np.dot(self.w, bf)

    # Calculate the model value for a vector
    def h(self, x: np.array) -> np.array:
        return np.array([self.hi(xi) for xi in x])

    def e(self) -> float:
        return np.sum((self.h(self.data[0]) - self.data[1]) ** 2) / 2

    def summary(self):
        print('Model: {}'.format(self.__class__.__name__))
        print('Error: {}'.format(round(self.e(), 2)))

    def train(self):
        for _ in range(self.epochs):
            self.save()
            self.train_step()

    def train_step(self):
        a = self.a
        x_ = self.data[0]
        y_ = self.data[1]
        for x, y in zip(x_, y_):
            b = self.basis_function(x)
            h = self.hi(x)
            for i in range(self.c):
                self.w[i] -= a * (h - y) * b[i]

    def save(self):
        self.error_history = np.append(self.error_history, self.e())

In [139]:
class ModelPolynomial(Model):
    def __init__(self, d: np.array):
        super().__init__(d)
        self.n: np.array = np.arange(self.c)

    def equation_basis_function(self) -> sp.Expr:
        x: sp.Symbol = sp.Symbol('x')
        n: sp.Symbol = sp.Symbol('n')
        return x ** n

    def equation(self) -> sp.Expr:
        r: sp.Expr = sp.sympify(0)
        for w, n in zip(self.w, self.n):
            w_: float = round(w, 2)
            r += w_ * self.equation_basis_function().subs({'n': n})
        return r

    def basis_function(self, x: float) -> np.array:
        x_: np.array = np.full(self.c, x)
        return x_ ** self.n


model_p: Model = ModelPolynomial(my_data)
model_p.summary()
model_p.equation_basis_function()

Model: ModelPolynomial
Error: 54.08


x**n

In [140]:
model_p.equation()

0.27*x**8 - 0.14*x**7 - 0.11*x**6 + 0.11*x**5 - 0.05*x**4 + 0.06*x**3 - 0.02*x**2 + 0.34*x + 0.38

In [141]:
class ModelGaussian(Model):
    def __init__(self, d: np.array):
        super().__init__(d)
        self.m: np.array = np.linspace(0, 1, self.c)
        self.s: np.array = np.full(self.c, 0.1)

    def equation_basis_function(self) -> sp.Expr:
        x: sp.Symbol = sp.Symbol('x')
        m: sp.Symbol = sp.Symbol('m')
        s: sp.Symbol = sp.Symbol('s')
        return sp.exp(-1 * ((x - m) ** 2) / (2 * (s ** 2)))

    def equation(self) -> sp.Expr:
        r: sp.Expr = sp.sympify(0)
        for w, m, s in zip(self.w, self.m, self.s):
            w_: float = round(w, 2)
            r += w_ * self.equation_basis_function().subs({'m': m, 's': s})
        return r

    def basis_function(self, x: float) -> np.array:
        x_: np.array = np.full(self.c, x)
        return np.exp(-1 * ((x_ - self.m) ** 2) / (2 * (self.s ** 2)))


model_g: Model = ModelGaussian(my_data)
model_g.summary()
model_g.equation_basis_function()

Model: ModelGaussian
Error: 64.66


exp(-(-m + x)**2/(2*s**2))

In [142]:
model_g.equation()

-0.47*exp(-50.0*(x - 0.125)**2) - 0.44*exp(-50.0*(x - 0.25)**2) - 0.27*exp(-50.0*(x - 0.375)**2) + 0.22*exp(-50.0*(x - 0.5)**2) - 0.2*exp(-50.0*(x - 0.625)**2) + 0.24*exp(-50.0*(x - 0.75)**2) + 0.28*exp(-50.0*(x - 0.875)**2) + 0.21*exp(-50.0*(x - 1.0)**2) + 0.35*exp(-50.0*x**2)

In [143]:
class ModelSigmoidal(Model):
    def __init__(self, d: np.array):
        super().__init__(d)
        self.m: np.array = np.linspace(0, 1, self.c)
        self.s: np.array = np.full(self.c, 0.1)

    def equation_basis_function(self) -> sp.Expr:
        x: sp.Symbol = sp.Symbol('x')
        m: sp.Symbol = sp.Symbol('m')
        s: sp.Symbol = sp.Symbol('s')
        return 1 / (1 + sp.exp((m - x) / s))

    def equation(self) -> sp.Expr:
        r: sp.Expr = sp.sympify(0)
        for w, m, s in zip(self.w, self.m, self.s):
            w_: float = round(w, 2)
            r += w_ * self.equation_basis_function().subs({'m': m, 's': s})
        return r

    def basis_function(self, x: float) -> np.array:
        x_: np.array = np.full(self.c, x)
        return 1 / (1 + np.exp((self.m - x_) / self.s))


model_s: Model = ModelSigmoidal(my_data)
model_s.summary()
model_s.equation_basis_function()

Model: ModelSigmoidal
Error: 30.89


1/(exp((m - x)/s) + 1)

In [144]:
model_s.equation()

-0.11/(1 + 22026.4657948067*exp(-10.0*x)) + 0.24/(1 + 6310.68810808902*exp(-10.0*x)) - 0.3/(1 + 1808.04241445606*exp(-10.0*x)) + 0.13/(1 + 518.012824668342*exp(-10.0*x)) + 0.02/(1 + 148.413159102577*exp(-10.0*x)) + 0.17/(1 + 42.5210820000628*exp(-10.0*x)) - 0.37/(1 + 12.1824939607035*exp(-10.0*x)) - 0.46/(1 + 3.49034295746184*exp(-10.0*x)) + 0.08/(1 + exp(-10.0*x))

In [145]:
fun['h_pol'] = model_p.h(fun['x'])
fun['h_gau'] = model_g.h(fun['x'])
fun['h_sig'] = model_s.h(fun['x'])

In [146]:
fig = py.graph_objs.Figure()
fig.add_scatter(x=fun['x'], y=fun['sin'], name=str(gen_function), line_dash='dash', line_color='gray')
fig.add_scatter(x=my_data[0], y=my_data[1], name='Points', mode='markers', marker_color='red')
fig.add_scatter(x=fun['x'], y=fun['h_pol'], name='Polynomial', line_color='blue')
fig.add_scatter(x=fun['x'], y=fun['h_gau'], name='Gaussian', line_color='green')
fig.add_scatter(x=fun['x'], y=fun['h_sig'], name='Sigmoidal', line_color='purple')
fig.show()

In [147]:
model_p.summary()
model_p.train()
model_p.summary()
model_p.equation()

Model: ModelPolynomial
Error: 54.08
Model: ModelPolynomial
Error: 6.97


0.95*x**8 + 0.42*x**7 + 0.28*x**6 + 0.25*x**5 - 0.26*x**4 - 0.6*x**3 - 1.18*x**2 - 0.93*x + 0.81

In [149]:
model_g.summary()
model_g.train()
model_g.summary()
model_g.equation()

Model: ModelGaussian
Error: 64.66
Model: ModelGaussian
Error: 1.33


0.21*exp(-50.0*(x - 0.125)**2) + 0.79*exp(-50.0*(x - 0.25)**2) + 0.26*exp(-50.0*(x - 0.375)**2) + 0.09*exp(-50.0*(x - 0.5)**2) - 0.55*exp(-50.0*(x - 0.625)**2) - 0.54*exp(-50.0*(x - 0.75)**2) - 0.44*exp(-50.0*(x - 0.875)**2) + 0.09*exp(-50.0*(x - 1.0)**2) + 0.14*exp(-50.0*x**2)

In [151]:
model_s.summary()
model_s.train()
model_s.summary()
model_s.equation()

Model: ModelSigmoidal
Error: 30.89
Model: ModelSigmoidal
Error: 3.1


0.35/(1 + 22026.4657948067*exp(-10.0*x)) + 0.79/(1 + 6310.68810808902*exp(-10.0*x)) - 0.19/(1 + 1808.04241445606*exp(-10.0*x)) - 0.49/(1 + 518.012824668342*exp(-10.0*x)) - 1.01/(1 + 148.413159102577*exp(-10.0*x)) - 0.6/(1 + 42.5210820000628*exp(-10.0*x)) - 0.35/(1 + 12.1824939607035*exp(-10.0*x)) + 0.27/(1 + 3.49034295746184*exp(-10.0*x)) + 1.07/(1 + exp(-10.0*x))

In [163]:
fig = py.graph_objs.Figure()
fig.add_scatter(y=model_p.error_history, name='Polynomial', line_color='blue')
fig.add_scatter(y=model_g.error_history, name='Gaussian', line_color='green')
fig.add_scatter(y=model_s.error_history, name='Sigmoidal', line_color='purple')
fig.show()

In [154]:
fun['h_pol'] = model_p.h(fun['x'])
fun['h_gau'] = model_g.h(fun['x'])
fun['h_sig'] = model_s.h(fun['x'])
fig = py.graph_objs.Figure()
fig.add_scatter(x=fun['x'], y=fun['sin'], name=str(gen_function), line_dash='dash', line_color='gray')
fig.add_scatter(x=my_data[0], y=my_data[1], name='Points', mode='markers', marker_color='red')
fig.add_scatter(x=fun['x'], y=fun['h_pol'], name='Polynomial', line_color='blue')
fig.add_scatter(x=fun['x'], y=fun['h_gau'], name='Gaussian', line_color='green')
fig.add_scatter(x=fun['x'], y=fun['h_sig'], name='Sigmoidal', line_color='purple')
fig.show()